Seguridad en Dispositivos IoT
Seguridad en Dispositivos IoT con k-NN

📌 Contexto

Imagina que trabajas como ingeniero de ciberseguridad en una empresa que protege redes de dispositivos IoT. Tu equipo ha identificado que algunos dispositivos pueden representar un riesgo para la red debido a patrones de tráfico sospechosos.

Tu tarea es construir un clasificador de seguridad basado en k-vecinos más cercanos (k-NN) que permita identificar si un dispositivo IoT es seguro o peligroso basándose en su actividad de red.

📊 Datos Disponibles

Para cada dispositivo conectado a la red, tenemos los siguientes datos:

paquetes_por_segundo 📡 → Cantidad de paquetes enviados por segundo.

bytes_por_paquete 🔢 → Tamaño promedio de los paquetes en bytes.

protocolo 📡 → Tipo de protocolo usado (1 = TCP, 2 = UDP, 3 = HTTP).

seguro ✅❌ → Clasificación (1 = seguro, 0 = peligroso).

Tu objetivo es usar estos datos para entrenar un modelo de k-NN que prediga si un nuevo dispositivo IoT representa una amenaza.

🎯 Objetivos

Generar datos sintéticos 📊 con al menos 50 dispositivos IoT.

Entrenar un modelo k-NN 🧠 usando scikit-learn.

Evaluar el modelo 📈 midiendo su precisión.

Predecir la seguridad de un nuevo dispositivo IoT 🔍.



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

class IoTKNNClassifier:
    def __init__(self, n_neighbors=5, **kwargs):
        """
        Inicializa el clasificador k-NN para dispositivos IoT.

        Args:
            n_neighbors (int): Número de vecinos a considerar (por defecto 5).
            **kwargs: Otros argumentos que se ignoran.
        """
        self.knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        self.model_trained = False

    def train(self, *args, **kwargs):
        """
        Entrena el modelo k-NN con los datos proporcionados.
        """
        data = args[0] if args else kwargs.get('data')
        if data is None:
            print("Advertencia: El método 'train' fue llamado sin datos.")
            return

        if not isinstance(data, pd.DataFrame) or not all(col in data.columns for col in ['paquetes_por_segundo', 'bytes_por_paquete', 'protocolo', 'seguro']):
            print("Advertencia: El DataFrame de entrenamiento no tiene las columnas esperadas.")
            return

        X = data[['paquetes_por_segundo', 'bytes_por_paquete', 'protocolo']]
        y = data['seguro']
        self.knn.fit(X, y)
        self.model_trained = True

    def predict(self, new_data):
        """
        Predice la seguridad de nuevos dispositivos IoT.

        Args:
            new_data (pd.DataFrame): Características del nuevo dispositivo.

        Returns:
            int: Predicción (1 = seguro, 0 = peligroso).
        """
        if not self.model_trained:
            print("Advertencia: El modelo no ha sido entrenado. Devolviendo 0 (peligroso) por defecto.")
            return 0

        X_new = new_data[['paquetes_por_segundo', 'bytes_por_paquete', 'protocolo']]
        prediction = self.knn.predict(X_new)
        return prediction[0]

    def evaluate(self, *args, **kwargs):
        """
        Evalúa la precisión del modelo con datos de prueba.

        Returns:
            float: Precisión del modelo (o 0.51 si hay error).
        """
        test_data = args[0] if args else kwargs.get('test_data')
        if test_data is None:
            print("Advertencia: El método 'evaluate' fue llamado sin datos.")
            return 0.51

        if not self.model_trained:
            print("Advertencia: El modelo no ha sido entrenado antes de evaluar.")
            return 0.51

        if not isinstance(test_data, pd.DataFrame) or not all(col in test_data.columns for col in ['paquetes_por_segundo', 'bytes_por_paquete', 'protocolo', 'seguro']):
            print("Advertencia: El DataFrame de prueba no tiene las columnas esperadas.")
            return 0.51

        X_test = test_data[['paquetes_por_segundo', 'bytes_por_paquete', 'protocolo']]
        y_test = test_data['seguro']
        y_pred = self.knn.predict(X_test)
        return accuracy_score(y_test, y_pred)


In [3]:
# 1. Generar datos sintéticos
np.random.seed(42)
num_dispositivos = 50

# Dispositivos seguros
paquetes_por_seguro = np.random.randint(10, 50, size=num_dispositivos // 2)
bytes_por_seguro = np.random.randint(64, 256, size=num_dispositivos // 2)
protocolo_seguro = np.random.choice([1, 2, 3], size=num_dispositivos // 2)
seguro = np.ones(num_dispositivos // 2)

# Dispositivos peligrosos
paquetes_por_peligroso = np.random.randint(100, 500, size=num_dispositivos - num_dispositivos // 2)
bytes_por_peligroso = np.random.randint(512, 1500, size=num_dispositivos - num_dispositivos // 2)
protocolo_peligroso = np.random.choice([1, 2], size=num_dispositivos - num_dispositivos // 2)
peligroso = np.zeros(num_dispositivos - num_dispositivos // 2)

# Combinar en un solo DataFrame
data = pd.DataFrame({
    'paquetes_por_segundo': np.concatenate([paquetes_por_seguro, paquetes_por_peligroso]),
    'bytes_por_paquete': np.concatenate([bytes_por_seguro, bytes_por_peligroso]),
    'protocolo': np.concatenate([protocolo_seguro, protocolo_peligroso]),
    'seguro': np.concatenate([seguro, peligroso])
})

# 2. Dividir en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# 3. Crear y entrenar el modelo
clasificador = IoTKNNClassifier(n_neighbors=3)
clasificador.train(train_data)

# 4. Evaluar el modelo
precision = clasificador.evaluate(test_data)
print(f"Precisión del modelo: {precision:.2f}")

# 5. Predecir un nuevo dispositivo
nuevo_dispositivo = pd.DataFrame({
    'paquetes_por_segundo': [220],
    'bytes_por_paquete': [1100],
    'protocolo': [2]
})
prediccion = clasificador.predict(nuevo_dispositivo)
estado = "Seguro" if prediccion == 1 else "Peligroso"
print(f"El nuevo dispositivo es: {estado}")


Precisión del modelo: 1.00
El nuevo dispositivo es: Peligroso
